In [466]:
import networkx as nx
from jellyfish import damerau_levenshtein_distance
from networkx import DiGraph,subgraph_view, connected_components, draw, bipartite_layout, is_connected
from networkx.classes.filters import hide_nodes
import matplotlib.pyplot as plt
from math import ceil, fmod

In [461]:
'''
DESCRIPTION 
    Utility function that chops a sequence into several reads with bounded random lengths that 
    have a bounded random overlap
INPUT
    sequence       | a sequence of characters that will be divided into overlapping subsequences
    min_subseq_len | the shortest length a subsequence can have
    max_subseq_len | the longest length a subsequence can have
    min_overlap    | the shortest overlap two subsequences can share
    max_overlap    | the longest overlap two subsequences can share
    circularize    | boolean indicating whether to add a random amount of the end of the sequence
                   | to the beginning and vice versa
    seed           | random seed for the random function for reproducibility
OUTPUT
    A list of overlapping reads of random bounded size which share a bounded random amount of
    overlap
'''
def generate_reads(sequence,min_subseq_len,max_subseq_len,min_overlap,max_overlap,min_coverage=None,circularise=False,seed=None):
    import random

    random.seed(seed)
    if circularise: sequence = sequence[-random.randint(min_overlap,max_overlap):] + sequence + sequence[:random.randint(min_overlap,max_overlap)]
    reads = []
    while 1: 
        start = 0
        end = random.randint(min_subseq_len,max_subseq_len)
        reads += [sequence[start:end]]
        while end < len(sequence):
            start = random.randint(end-max_overlap,end-min_overlap)
            if (len(sequence) - start)/max_subseq_len < 2:
                if (len(sequence) - start)/max_subseq_len < 1:
                    end = len(sequence)
                else:
                    a = 0
                    while (len(sequence) - start)/(min_subseq_len+a) > 2: a+=1
                    end = random.randint(start+min_subseq_len+a,start+max_subseq_len) 
            else: end = random.randint(start+min_subseq_len,start+max_subseq_len) 
            reads += [sequence[start:end]]
        if min_coverage is None or len(set(reads))*(sum(len(read) for read in set(reads))/len(set(reads)))/len(sequence) >= min_coverage: return reads
        # if min_coverage is None or len(set(reads))*(sum(len(read) for read in set(reads))/len(set(reads)))/len(sequence) >= min_coverage: return list(set(reads))

'''
DESCRIPTION 
    Utility function that creates a random sequence containing only the letters A, T, G, and C
INPUT
    n          | the length of the sequence
    palindrome | a boolean indicating whether the sequence must be a palidrome or not
    seed       | random seed for the random function for reproducibility
OUTPUT
    A random sequence of length n
'''
def generate_genome_sequence(n,palindrome=False,seed=None):
    import random
    
    random.seed(seed)
    nucleotides = {1:'A',2:'C',3:'G',4:'T'}
    seq = ''
    if palindrome: n = ceil(n/2)
    for _ in range(n):
        seq += nucleotides[random.randint(1,4)]
    if palindrome: seq += ''.join(reversed(seq[:int(n-fmod(n,2))]))
    return seq

# Sequitur

In [667]:
def normalised_damerau_levenshtein_distance(read,overlap):
    return damerau_levenshtein_distance(read.__str__()[:min(len(overlap),len(read))],overlap.__str__()[:min(len(overlap),len(read))])/min(len(overlap),len(read))

def build_suffix_array(reads,min_suf_len=3):
    suf_arr = []
    for read in reads:
        read += '$' + str(reads.index(read))
        for i in range(len(read)):
            if len(read[i:]) < min_suf_len + 2: continue 
            suf_arr += [read[i:]]
    suf_arr.sort()
    suf_arr_ind = []
    for s in range(len(suf_arr)):
        suf_arr_ind += [int(suf_arr[s].split('$')[-1].__str__())]
        suf_arr[s] = suf_arr[s][:suf_arr[s].find('$')+1]
    return suf_arr,suf_arr_ind

def build_bipartite_graph(reads,suf_arr=None,suf_arr_ind=None,max_diff=0.25,min_suf_len=3):
    if suf_arr is None or suf_arr_ind is None: suf_arr,suf_arr_ind = build_suffix_array(reads)
    B = DiGraph()
    for read in reads:
        for j in range(min_suf_len + 1):
            i = suf_arr.index(read[j:]+'$') - 1
            while normalised_damerau_levenshtein_distance(read,suf_arr[i][:-1]) <= 0.5:
                if not reads[suf_arr_ind[i]] == read and \
                   not B.has_edge('^' + reads[suf_arr_ind[i]],read + '$') and \
                   normalised_damerau_levenshtein_distance(read,suf_arr[i][:-1]) < max_diff and \
                   read.startswith(suf_arr[i][:-1]):
                        B.add_node('^' + reads[suf_arr_ind[i]],bipartite=0) 
                        B.add_node(read + '$',bipartite=1) 
                        B.add_edge('^' + reads[suf_arr_ind[i]],read + '$',weight=len(suf_arr[i][:-1]),rejected=False)
                i -= 1
    return B

def reset_edges(B):
    for edge in B.edges(): B[edge[0]][edge[1]]['rejected'] = False

def random_initial_edge(G,B,seed=None):
    import random
    random.seed(seed)

    if len(G):
        B = subgraph_view(B,filter_node=hide_nodes(list(s+'$' for s,d in G.in_degree if d == 1)))
        B = subgraph_view(B,filter_node=hide_nodes(list('^'+p for p,d in G.out_degree if d == 1)))
        B = subgraph_view(B,filter_node=hide_nodes(list(n for n,d in B.degree if d == 0)))
        reset_edges(B)
    if len(list(e for e, d in B.nodes(data=True) if d["bipartite"] == 0)):
        p = random.choice(list(e for e, d in B.nodes(data=True) if d["bipartite"] == 0))
        s = sorted(list(B[p].items()),key=lambda e: e[1]['weight'],reverse=True)[0][0]
        G.add_node(p[1:],successor=s[:-1])
        G.add_node(s[:-1],predecessor=p[1:])
        G.add_edge(p[1:],s[:-1])
        return (p,s)

def sequence(G,B):
    n = [node for node in G.nodes() if G.in_degree(node)==0][0]
    seq = n
    i = 1
    while 'successor' in G.nodes[n]:
        seq += G.nodes[n]['successor'][B['^'+n][G.nodes[n]['successor']+'$']['weight']:]
        n = G.nodes[n]['successor']
        i += 1
    return seq

def is_disconnected(G,node1,node2):
    for c in connected_components(G.to_undirected()):
        if (node1 not in c and node2 in c) or (node1 in c and node2 not in c): return True
    return False


def sequitur(G,B,reads,edge,seed=None,true_sequence=None):
    import random

    random.seed(seed)
    state_stack = []
    while len(G.nodes) < len(reads) or not is_connected(G.to_undirected()):
        b = False
        if ('^'+edge[1][:-1] not in B or all(B.edges[('^'+edge[1][:-1],e)]['rejected'] for e in B['^'+edge[1][:-1]]) or 'successor' in G.nodes[edge[1][:-1]]) and len(G.nodes) < len(reads):
            edge = random_initial_edge(G,B,seed=seed)
            continue
        for _,successor in sorted(list(('^'+edge[1][:-1],e) for e in B['^'+edge[1][:-1]] if not B.edges[('^'+edge[1][:-1],e)]['rejected']),key=lambda e: B.edges[e]['weight'],reverse=True):
            if successor[:-1] not in G:
                edge = ('^' + edge[1][:-1],successor)
                G.add_node(edge[0][1:],successor=edge[1][:-1])
                G.add_node(edge[1][:-1],predecessor=edge[0][1:])
                G.add_edge(edge[0][1:],edge[1][:-1])
                b = True
                break
            elif is_disconnected(G,successor[:-1],edge[1][:-1]) and G.in_degree(successor[:-1])==0:
                edge = ('^' + edge[1][:-1],successor)
                G.add_node(edge[0][1:],successor=edge[1][:-1])
                G.add_node(edge[1][:-1],predecessor=edge[0][1:])
                G.add_edge(edge[0][1:],edge[1][:-1])
                b = True
                if len(G.nodes) < len(reads): edge = random_initial_edge(G,B,seed=seed)
                break
            else: B.edges[('^' + edge[1][:-1],successor)]['rejected'] = True
        if '^'+edge[1][:-1] not in B or b: continue
        if all(B.edges[('^'+edge[1][:-1],e)]['rejected'] for e in B['^'+edge[1][:-1]]):
            # bb = False
            # while B.edges[edge]['rejected'] or not b:
            if any('predecessor' in G.nodes[e[:-1]] for e in B['^'+edge[1][:-1]] if e[:-1] in G):# or bb:  
                # if bb:
                #     state = state_stack.pop()
                #     G = state['G']
                #     B = state['B']
                #     p = state['p']
                #     bb = False
                # else:
                p = list(('^'+G.nodes[e[:-1]]['predecessor'],e) 
                                    for e in B['^'+edge[1][:-1]] 
                                        if (e[:-1] in G and \
                                            'predecessor' in G.nodes[e[:-1]]))# and \
                                            #  ('^'+G.nodes[e[:-1]]['predecessor'],e) not in exclude)
                p.sort(key=lambda e: B.edges[e]['weight'],reverse=True)
                for e in B['^'+edge[1][:-1]]:  B.edges[('^'+edge[1][:-1],e)]['rejected'] = False
                edge = p.pop(0)
                # exclude.add(edge)
                # if len(p):
                #     state_stack += [{
                #         'G': G.copy(),
                #         'B': B.copy(),
                #         'p': p[:]
                #     }]
                r = [edge[1][:-1]]
                while 'successor' in G.nodes[r[-1]]: 
                    for endpoint in B['^'+r[-1]]: B.edges[('^'+r[-1],endpoint)]['rejected'] = False
                    r.append(G.nodes[r[-1]]['successor'])
                r.pop(0)
                G.remove_nodes_from(r)
            if not G.out_degree[edge[1][:-1]]: G.remove_node(edge[1][:-1])
            else: 
                G.remove_edge(edge[0][1:],edge[1][:-1])
                if "predecessor" in G.nodes[edge[1][:-1]]: G.nodes[edge[1][:-1]].pop("predecessor")
            G.nodes[edge[0][1:]].pop("successor")
            B.edges[edge]['rejected'] = True
            if G.in_degree[edge[0][1:]]: 
                edge = ('^'+G.nodes[edge[0][1:]]["predecessor"],edge[0][1:]+'$')
                b = True
            elif any(not B.edges[(edge[0],e)]['rejected'] for e in B[edge[0]]):
                if len(G.nodes) < len(reads): edge = (edge[0],sorted(list({e for e in set(B[edge[0]]).difference({node+'$' for node,degree in G.in_degree() if degree > 0}) 
                                                                        if not B.edges[(edge[0],e)]['rejected']}),key=lambda e: B.edges[(edge[0],e)]['weight'],reverse=True)[0])
                G.add_node(edge[0][1:],successor=edge[1][:-1])
                G.add_node(edge[1][:-1],predecessor=edge[0][1:])
                G.add_edge(edge[0][1:],edge[1][:-1])
                b = True
            else:
                B.edges[edge]['rejected'] = False
                G.add_node(edge[0][1:],successor=edge[1][:-1])
                G.add_node(edge[1][:-1],predecessor=edge[0][1:])
                G.add_edge(edge[0][1:],edge[1][:-1])
                b = True
    return sequence(G,B)

def draw_bipartite(B,reads):
    _, ax = plt.subplots(figsize=(12, 12))
    draw(B,
            pos=bipartite_layout(B,set('^' + read for read in reads).intersection(B.nodes())),
            node_size=100,
            font_size=12,
            with_labels=True,
            bbox={"ec": "k", "fc": "white", "alpha": 0.7}
            )
    ax.margins(0.15, 0.05)
    plt.show()

In [625]:
seq = 'betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better'
reads = ['betty_bought_butter_th',
                        'tter_the_butter_was_',
                              'he_butter_was_bitter_',
                                         'as_bitter_betty_bought',
                                                     'tty_bought_better_butter_t',
                                                           'ught_better_butter_to',
                                                                     'r_butter_to_make_the_',
                                                                                   'ke_the_bitter_butter_better']

seed= 0
B = build_bipartite_graph(reads)
G = nx.DiGraph()
sequitur(G,B,reads,random_initial_edge(G,B,seed=seed),seed=seed)

'betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better'

In [626]:
seq = 'you say hello world, i bellow go to hell'
reads = ['you say hel',
            ' say hello wo',
                    'lo world, i be',
                          'ld, i bellow go t',
                                    'ow go to hell']
seed = 0
B = build_bipartite_graph(reads)
G = nx.DiGraph()
sequitur(G,B,reads,random_initial_edge(G,B,seed=seed),seed=seed)

'you say hello world, i bellow go to hell'

In [627]:
seq = 'she_sells_sea_shells_on_the_sea_shore'
reads = ['she_sells_s',
               'lls_sea_shel',
                    'ea_shells_o',
                       'shells_on_the_s',
                                  'he_sea_s',
                                      'ea_shore']
seed = 0
B = build_bipartite_graph(reads)
G = nx.DiGraph()
sequitur(G,B,reads,random_initial_edge(G,B,seed=seed),seed=seed)

'she_sells_sea_shells_on_the_sea_shore'

In [669]:
# REQUIRES CONNECTEDNESS CHECK
successes = 0
n = 50
for seed in range(n):  
    seq = generate_genome_sequence(10000,seed=seed)
    reads = generate_reads(seq,250,500,50,100,seed=seed)
    B = build_bipartite_graph(reads)
    G = nx.DiGraph()
    seq_ = sequitur(G,B,reads,random_initial_edge(G,B,seed=seed),seed=seed)
    
    s = '| Seed: ' + str(seed) + ' | '
    if seq_ == seq:
        s+='SUC | ' + seq_ + ' == ' + seq
        successes+=1
    else: 
        s+='FAI | ' + seq_ + ' != ' + seq
        print(s)
        break
    print(s)
    print('-----------------------------------------')
print('ACCURACY: '+str((successes/n)*100)+'%')

| Seed: 0 | SUC | ...
-----------------------------------------
| Seed: 1 | SUC | ...
-----------------------------------------
| Seed: 2 | SUC | ...
-----------------------------------------
| Seed: 3 | SUC | ...
-----------------------------------------
| Seed: 4 | SUC | ...
-----------------------------------------
| Seed: 5 | SUC | ...
-----------------------------------------
| Seed: 6 | SUC | ...
-----------------------------------------
| Seed: 7 | SUC | ...
-----------------------------------------
| Seed: 8 | SUC | ...
-----------------------------------------
| Seed: 9 | SUC | ...
-----------------------------------------
| Seed: 10 | SUC | ...
-----------------------------------------
| Seed: 11 | SUC | ...
-----------------------------------------
| Seed: 12 | SUC | ...
-----------------------------------------
| Seed: 13 | SUC | ...
-----------------------------------------
| Seed: 14 | SUC | ...
-----------------------------------------
| Seed: 15 | SUC | ...
------------

    SUC: returns the target sequence fully reconstructed
    PAR: returns contigs all of which exist in the target sequence (consider coverage?)
    FAI: returns a full sequence that is incorrectly reconstructed or a set of contigs where at least one is not found in the target sequence

In [215]:
#! pip install Bio
from Bio import SeqIO

# B_ = {}

In [649]:
seed = 0
for record in SeqIO.parse("data/input/Raphanus sativus_NC_018551.1.fasta",'fasta'): seq = record.seq
reads = generate_reads(seq,250,250,50,50,seed=seed,min_coverage=None) 
if seed in B_:
    B = B_[seed].copy() 
else:
    B = build_bipartite_graph(reads)
    B_[seed] = B.copy()
G = nx.DiGraph()
sequitur(G,B,reads,random_initial_edge(G,B,seed=seed),seed=seed,true_sequence=seq) == seq

True

In [670]:
seed = 1
for record in SeqIO.parse("data/input/Raphanus sativus_NC_018551.1.fasta",'fasta'): seq = record.seq
reads = generate_reads(seq,250,250,50,50,seed=seed,min_coverage=None) 
if seed in B_:
    B = B_[seed].copy() 
else:
    B = build_bipartite_graph(reads)
    B_[seed] = B.copy()
G = nx.DiGraph()
sequitur(G,B,reads,random_initial_edge(G,B,seed=seed),seed=seed,true_sequence=seq) == seq

True